<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/from_gee_to_colab_archeology_6june2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Install geemap if not already installed
!pip install geemap



In [ ]:
# based on similar date
import geemap

# Define the region of interest (Eastern Brazil)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()

# Clip to Eastern Brazil using longitude filtering
#east_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-40, -5, -30, -30]))  # Adjust as needed
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))  # Adjust as needed
# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-05-01'
end_date = '2024-05-31'
def classify_image(date):
    date = ee.Date(date)

    sentinel2 = mosaiced_sentinel2.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()
    sentinel1 = mosaiced_sentinel1.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()

    combined = sentinel2.addBands(sentinel1)

    classified = ee.Algorithms.If(
        archaeology_points.size().gt(0),
        combined.classify(
            ee.Classifier.smileRandomForest(10).train(
                combined.sampleRegions(collection=archaeology_points, properties=['class'], scale=30),
                'class'
            )
        ),
        ee.Image.constant(-9999).rename("classification")  # Placeholder if no training points
    )

    return ee.Image(classified).set("date", date.format("YYYY-MM-dd"))
### Step 1: Mosaic Sentinel-1 images per date ###
def mosaic_sentinel1(date):
    date = ee.Date(date)
    sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(southeast_brazil) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV")

    return sentinel1_images.median().set("date", date.format("YYYY-MM-dd"))

### Step 2: Mosaic Sentinel-2 images per date ###
def mosaic_sentinel2(date):
    date = ee.Date(date)
    sentinel2_images = ee.ImageCollection("COPERNICUS/S2").filterBounds(southeast_brazil) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(["B8", "B11", "B12"])

    return sentinel2_images.median().set("date", date.format("YYYY-MM-dd"))

# Extract available dates
sentinel1_dates = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .aggregate_array('system:time_start').map(ee.Date)

sentinel2_dates = ee.ImageCollection("COPERNICUS/S2").filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .aggregate_array('system:time_start').map(ee.Date)

# Find matching dates between Sentinel-1 and Sentinel-2
formatted_sentinel1_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
formatted_sentinel2_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))

common_dates = formatted_sentinel1_dates.filter(ee.Filter.inList('item', formatted_sentinel2_dates))

# Create mosaiced collections
mosaiced_sentinel1 = ee.ImageCollection(common_dates.map(mosaic_sentinel1))
mosaiced_sentinel2 = ee.ImageCollection(common_dates.map(mosaic_sentinel2))

# Create Map
m = geemap.Map(center=[-20, -40], zoom=5)  # Centered around Eastern Brazil

# Display Sentinel-1 mosaiced images
list_s1 = mosaiced_sentinel1.toList(mosaiced_sentinel1.size())
for i in range(list_s1.size().getInfo()):
    img = ee.Image(list_s1.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
    m.addLayer(img, vis_params_s1, f"Sentinel-1 Mosaiced ({date_label})")

# Display Sentinel-2 mosaiced images
list_s2 = mosaiced_sentinel2.toList(mosaiced_sentinel2.size())
for i in range(list_s2.size().getInfo()):
    img = ee.Image(list_s2.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
    m.addLayer(img, vis_params_s2, f"Sentinel-2 Mosaiced ({date_label})")
'''classified_images = ee.ImageCollection(common_dates.map(classify_image))
print("Number of classified images:", classified_images.size().getInfo())
# Display the map
# Display Classified Images
list_classified = classified_images.toList(classified_images.size())
for i in range(list_classified.size().getInfo()):
    img = ee.Image(list_classified.get(i))
    date_label = img.get("date").getInfo()
    vis_params_classified = {
        "bands": ["classification"],
        "min": 0,
        "max": 1,
        "palette": ["red", "yellow", "green"]
    }
    m.addLayer(img, vis_params_classified, f"Classified Archaeological Predictions ({date_label})")'''
# Add the archaeology points layer
archaeology_vis = {
    'color': 'blue',
    'pointRadius': 5
}
m.addLayer(archaeology_points, archaeology_vis, 'Archaeology Points')

# Display the map
m


In [ ]:
!pip install /content/ee-packages-py-main/

In [ ]:
# Based on similar location
import geemap

# Define region (Brazil, Southeastern Amazon)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))

# Load archaeological points
archaeology_points = ee.FeatureCollection("users/mohamadawadlebanon/Archeologicalsites")

# Define date range
start_date = "2024-05-01"
end_date = "2024-05-31"
def apply_scale_and_offset(image):
    return image.select("ST_B10").multiply(0.00341802).add(149.0)
### **Step 1: Elevation Data (ASTER GDEM)**
elevation = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(southeast_brazil)
#elevation = elevation.rename("Elevation")
elevation = elevation.rename("elevation")
### **Step 2: Sentinel-2 EVI**
sentinel2_ndvi = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B4"]) \
    .map(lambda img: img.expression(
        "(B8 - B4) / (B8 + B4)",  # EVI formula
        {"B8": img.select("B8"), "B4": img.select("B4")}
    )).median().clip(southeast_brazil)
sentinel2_ndvi = sentinel2_ndvi.rename("NDVI")

### **Step 3: Sentinel-1 Radar**
sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .select("VV")

mosaiced_sentinel1 = sentinel1_images.median().clip(southeast_brazil)

### **Step 4: Sentinel-2 Optical Mosaic**
sentinel2_images = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B11", "B12"])

mosaiced_sentinel2 = sentinel2_images.median().clip(southeast_brazil)

### **Step 5: MODIS Evapotranspiration (ET)**
modis_et = ee.ImageCollection("MODIS/061/MOD16A2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select("ET") \
    .median().clip(southeast_brazil)

'''### **Step 6: Penman-Monteith-Leuning Evapotranspiration (PML-V2)**
pml_et = ee.ImageCollection("CAS/IGSNRR/PML/V2_v018") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select("ET") \
    .median().clip(southeast_brazil)'''

### **Step 7: Soil Moisture (NASA SMAP)**
soil_moisture = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select('sm_surface') \
    .median().clip(southeast_brazil)

### **Step 8: Thermal Infrared (Landsat 8-9)**
thermal = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .map(apply_scale_and_offset) \
    .median().clip(southeast_brazil)

### **Step 9: Combine Layers for Classification**
combined = mosaiced_sentinel2.addBands(mosaiced_sentinel1) \
    .addBands(elevation).addBands(sentinel2_ndvi).addBands(modis_et) \
    .addBands(soil_moisture).addBands(thermal)

# Sample feature values at archaeology site locations
sampled_values = combined.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 10,  # Adjust scale depending on resolution needs
    #"properties": ["site_id"],  # Add relevant properties
    "tileScale": 2
})
# Convert sampled values to FeatureCollection table
def safe_set_coordinates(feature):
    return feature.set({
        "Longitude": feature.get("lon"),
        "Latitude": feature.get("lat")
    })

#table = sampled_values.map(safe_set_coordinates)

task = ee.batch.Export.table.toDrive(
    collection=sampled_values, #table,
    description="Archaeological_Site_Features",
    fileFormat="CSV"
)
task.start()  # Start export task

### **Step 10: Apply K-Means Clustering**
num_classes = 10
'''training_points = combined.sample(**{
    "region": southeast_brazil,
    "scale": 10,
    "numPixels": 500,
    "seed": 42
})'''
training_points = combined.sample(
    region=southeast_brazil,
    # Default (False) is no geometries in the output.
    # When set to True, each feature has a Point geometry at the center of the
    # image pixel.
    geometries=True,
    numPixels = 500,
    # The scale is not specified, so the resolution of the image will be used,
    # and there is a feature for every pixel. If we give a scale parameter, the
    # image will be resampled and there will be more or fewer features.
    #
    scale=10,
)
# Extract longitude and latitude from geometry
'''training_points = training_points.map(lambda feature:
                     feature.set({
                         "Longitude": feature.geometry().coordinates().get(0),
                         "Latitude": feature.geometry().coordinates().get(1)
                     })

    )'''
#training_points = training_points.filter(ee.Filter.notNull(['geometry']))
print(training_points.first().getInfo())
task1 = ee.batch.Export.table.toDrive(
    collection=training_points,
    description="Samples_random_collection",
    fileFormat="CSV"
)
task1.start()
clusterer = ee.Clusterer.wekaKMeans(num_classes).train(training_points)
classified = combined.cluster(clusterer)

### **Step 11: Validate Using Archaeological Sites**
validation = classified.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 10,
    "properties": ["class"],
    "tileScale": 2
})

### **Step 12: Visualization Parameters**
vis_params_elevation = {"bands": ["elevation"], "min": 0, "max": 3000, "palette": ["black", "white"]}
vis_params_ndvi = {"bands": ["NDVI"],"min": -0.6, "max": 0.6, "palette": ["brown", "green"]}
vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
vis_params_modis_et = {"bands": ["ET"], "min": 0, "max": 300, "palette": ["yellow", "green", "blue"]}
#vis_params_pml_et = {"min": 0, "max": 5, "palette": ["orange", "red", "purple"]}
vis_params_soil = {"bands": ["sm_surface"], "min": 0, "max": 0.9, "palette": ["red", "orange", "yellow", "green"]}
vis_params_thermal = {"bands": ["ST_B10"], "min": 270, "max": 320, "palette": ["blue", "yellow", "red"]}
vis_params_classified = {
    "min": 0,
    "max": num_classes - 1,
    "palette": ["blue", "green", "yellow", "red", "purple", "orange", "brown", "cyan", "pink", "gray"]
}
archaeology_vis = {"color": "blue", "pointRadius": 5}
sample_vis = {"color": "black", "pointRadius": 4}
### **Step 13: Create & Display Map**
m = geemap.Map(center=[-3, -60], zoom=6)

m.addLayer(elevation, vis_params_elevation, "ASTER GDEM v3 Elevation")
m.addLayer(sentinel2_ndvi, vis_params_ndvi, "High-Resolution NDVI (Sentinel-2)")
m.addLayer(mosaiced_sentinel1, vis_params_s1, "Sentinel-1 Mosaic (Radar)")
m.addLayer(mosaiced_sentinel2, vis_params_s2, "Sentinel-2 Mosaic (Optical)")
m.addLayer(modis_et, vis_params_modis_et, "MODIS Evapotranspiration")
#m.addLayer(pml_et, vis_params_pml_et, "PML Evapotranspiration")
m.addLayer(soil_moisture, vis_params_soil, "Soil Moisture (SMAP)")
m.addLayer(thermal, vis_params_thermal, "Thermal Infrared (Landsat)")
m.addLayer(classified, vis_params_classified, "Classified Image (K-Means)")
m.addLayer(archaeology_points, archaeology_vis, "Archaeology Points")
m.addLayer(training_points,sample_vis, "sample Points")

# Display the map
m


In [ ]:
print(training_points.size().getInfo())  # Ensure points were actually sampled



In [ ]:
# Based on similar location - modified one to include weights
import geemap

# Define region (Brazil, Southeastern Amazon)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))
weights = {
    "B12": 0.3,
    "B11": 0.25,
    "B8": 0.2,
    "VV": 0.15,
    #"NDVI": 0.1,
    #"Elevation": 0.2,
    #"SoilMoisture": 0.15,
    #"Thermal": 0.2,
    #"Evapotranspiration": 0.1
}
def normalize_band(image, band):
    band_min = image.select(band).reduceRegion(
        reducer=ee.Reducer.min(), geometry=image.geometry(), scale=30, bestEffort=True
    ).get(band)

    band_max = image.select(band).reduceRegion(
        reducer=ee.Reducer.max(), geometry=image.geometry(), scale=30, bestEffort=True
    ).get(band)

    normalized_band = image.select(band).subtract(ee.Number(band_min)).divide(ee.Number(band_max).subtract(ee.Number(band_min)))
    return normalized_band.rename(band + "_norm")
# Load archaeological points
archaeology_points = ee.FeatureCollection("users/mohamadawadlebanon/Archeologicalsites")

# Define date range
start_date = "2024-05-01"
end_date = "2024-05-31"
def apply_scale_and_offset(image):
    return image.select("ST_B10").multiply(0.00341802).add(149.0)
### **Step 1: Elevation Data (ASTER GDEM)**
elevation = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(southeast_brazil)
#elevation = elevation.rename("Elevation")
elevation = elevation.rename("elevation")
### **Step 2: Sentinel-2 EVI**
sentinel2_ndvi = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B4"]) \
    .map(lambda img: img.expression(
        "(B8 - B4) / (B8 + B4)",  # EVI formula
        {"B8": img.select("B8"), "B4": img.select("B4")}
    )).median().clip(southeast_brazil)
sentinel2_ndvi = sentinel2_ndvi.rename("NDVI")

### **Step 3: Sentinel-1 Radar**
sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .select("VV")

mosaiced_sentinel1 = sentinel1_images.median().clip(southeast_brazil)

### **Step 4: Sentinel-2 Optical Mosaic**
sentinel2_images = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B11", "B12"])

mosaiced_sentinel2 = sentinel2_images.median().clip(southeast_brazil)
'''normalized_sentinel2 = ee.Image.cat(
    normalize_band(mosaiced_sentinel2, "B12"),
    normalize_band(mosaiced_sentinel2, "B11"),
    normalize_band(mosaiced_sentinel2, "B8")
)'''

### **Step 5: MODIS Evapotranspiration (ET)**
modis_et = ee.ImageCollection("MODIS/061/MOD16A2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select("ET") \
    .median().clip(southeast_brazil)

'''### **Step 6: Penman-Monteith-Leuning Evapotranspiration (PML-V2)**
pml_et = ee.ImageCollection("CAS/IGSNRR/PML/V2_v018") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select("ET") \
    .median().clip(southeast_brazil)'''

### **Step 7: Soil Moisture (NASA SMAP)**
soil_moisture = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select('sm_surface') \
    .median().clip(southeast_brazil)

### **Step 8: Thermal Infrared (Landsat 8-9)**
thermal = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .map(apply_scale_and_offset) \
    .median().clip(southeast_brazil)


# Normalize all bands
#normalized_sentinel2 = mosaiced_sentinel2.select(["B12", "B11", "B8"]).map(lambda img: normalize_band(mosaiced_sentinel2, img))
normalized_sentinel1 = normalize_band(mosaiced_sentinel1, "VV")
normalized_ndvi = normalize_band(sentinel2_ndvi, "NDVI")
normalized_elevation = normalize_band(elevation, "elevation")
normalized_soil_moisture = normalize_band(soil_moisture, "sm_surface")
normalized_thermal = normalize_band(thermal, "ST_B10")
normalized_et = normalize_band(modis_et, "ET")

### **Step 9: Combine Layers for Classification**
'''weighted_combined = (normalized_sentinel2.select("B12_norm").multiply(weights["B12"])
    .add(normalized_sentinel2.select("B11_norm").multiply(weights["B11"]))
    .add(normalized_sentinel2.select("B8_norm").multiply(weights["B8"]))
    .add(normalized_sentinel1.multiply(weights["VV"]))
    .add(normalized_ndvi.multiply(weights["NDVI"]))
    .add(normalized_elevation.multiply(weights["Elevation"]))
    .add(normalized_soil_moisture.multiply(weights["SoilMoisture"]))
    .add(normalized_thermal.multiply(weights["Thermal"]))
    .add(normalized_et.multiply(weights["Evapotranspiration"])
    .rename("Weighted_Combined"))'''
#combined = normalized_sentinel2.select("B12_norm").multiply(weights["B12"]).addBands(normalized_sentinel2.select("B11_norm").multiply(weights["B11"])).addBands(normalized_sentinel2.select("B8_norm").multiply(weights["B8"])).addBands(normalized_sentinel1.multiply(weights["VV"])).addBands(normalized_ndvi.multiply(weights["NDVI"])).addBands(normalized_elevation.multiply(weights["Elevation"])).addBands(normalized_soil_moisture.multiply(weights["SoilMoisture"])).addBands(normalized_thermal.multiply(weights["Thermal"])).addBands(normalized_et.multiply(weights["Evapotranspiration"]))
#combined = mosaiced_sentinel2.select("B12").multiply(weights["B12"]).addBands(mosaiced_sentinel2.select("B11").multiply(weights["B11"])).addBands(mosaiced_sentinel2.select("B8").multiply(weights["B8"])).addBands(mosaiced_sentinel1.multiply(weights["VV"])).addBands(sentinel2_ndvi.multiply(weights["NDVI"])).addBands(elevation.multiply(weights["Elevation"])).addBands(soil_moisture.multiply(weights["SoilMoisture"])).addBands(thermal.multiply(weights["Thermal"])).addBands(modis_et.multiply(weights["Evapotranspiration"]))
#combined = (sentinel2_ndvi.multiply(weights["NDVI"])).addBands(elevation.multiply(weights["Elevation"])).addBands(soil_moisture.multiply(weights["SoilMoisture"])).addBands(thermal.multiply(weights["Thermal"])).addBands(modis_et.multiply(weights["Evapotranspiration"]))
combined = mosaiced_sentinel2.select("B12").multiply(weights["B12"]).addBands(mosaiced_sentinel2.select("B11").multiply(weights["B11"])).addBands(mosaiced_sentinel2.select("B8").multiply(weights["B8"])).addBands(mosaiced_sentinel1.multiply(weights["VV"]))
'''combined1 = mosaiced_sentinel2.addBands(mosaiced_sentinel1) \
    .addBands(elevation).addBands(sentinel2_ndvi).addBands(modis_et) \
    .addBands(soil_moisture).addBands(thermal) \
    .addBands(weighted_combined)'''


# Sample feature values at archaeology site locations
sampled_values = combined.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 5,  # Adjust scale depending on resolution needs
    #"properties": ["site_id"],  # Add relevant properties
    "tileScale": 2
})
# Convert sampled values to FeatureCollection table
def safe_set_coordinates(feature):
    return feature.set({
        "Longitude": feature.get("lon"),
        "Latitude": feature.get("lat")
    })

#table = sampled_values.map(safe_set_coordinates)

task = ee.batch.Export.table.toDrive(
    collection=sampled_values, #table,
    description="Archaeological_Site_Features",
    fileFormat="CSV"
)
task.start()  # Start export task

### **Step 10: Apply K-Means Clustering**
num_classes = 10
'''training_points = combined.sample(**{
    "region": southeast_brazil,
    "scale": 10,
    "numPixels": 500,
    "seed": 42
})'''
training_points = combined.sample(
    region=southeast_brazil,
    # Default (False) is no geometries in the output.
    # When set to True, each feature has a Point geometry at the center of the
    # image pixel.
    geometries=True,
    numPixels = 500,
    # The scale is not specified, so the resolution of the image will be used,
    # and there is a feature for every pixel. If we give a scale parameter, the
    # image will be resampled and there will be more or fewer features.
    #
    scale= 10,
)
# Extract longitude and latitude from geometry
'''training_points = training_points.map(lambda feature:
                     feature.set({
                         "Longitude": feature.geometry().coordinates().get(0),
                         "Latitude": feature.geometry().coordinates().get(1)
                     })

    )'''
#training_points = training_points.filter(ee.Filter.notNull(['geometry']))
print(training_points.first().getInfo())
task1 = ee.batch.Export.table.toDrive(
    collection=training_points,
    description="Samples_random_collection",
    fileFormat="CSV"
)
task1.start()
clusterer = ee.Clusterer.wekaKMeans(num_classes).train(training_points)
classified = combined.cluster(clusterer)

### **Step 11: Validate Using Archaeological Sites**
validation = classified.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 10,
    "properties": ["class"],
    "tileScale": 2
})

### **Step 12: Visualization Parameters**
vis_params_elevation = {"bands": ["elevation"], "min": 0, "max": 3000, "palette": ["black", "white"]}
vis_params_ndvi = {"bands": ["NDVI"],"min": -0.6, "max": 0.6, "palette": ["brown", "green"]}
vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
vis_params_modis_et = {"bands": ["ET"], "min": 0, "max": 300, "palette": ["yellow", "green", "blue"]}
#vis_params_pml_et = {"min": 0, "max": 5, "palette": ["orange", "red", "purple"]}
vis_params_soil = {"bands": ["sm_surface"], "min": 0, "max": 0.9, "palette": ["red", "orange", "yellow", "green"]}
vis_params_thermal = {"bands": ["ST_B10"], "min": 270, "max": 320, "palette": ["blue", "yellow", "red"]}
vis_params_classified = {
    "min": 0,
    "max": num_classes - 1,
    "palette": ["blue", "green", "yellow", "red", "purple", "orange", "brown", "cyan", "pink", "gray"]
}
archaeology_vis = {"color": "blue", "pointRadius": 5}
sample_vis = {"color": "black", "pointRadius": 4}
### **Step 13: Create & Display Map**
m = geemap.Map(center=[-3, -60], zoom=6)

m.addLayer(elevation, vis_params_elevation, "ASTER GDEM v3 Elevation")
m.addLayer(sentinel2_ndvi, vis_params_ndvi, "High-Resolution NDVI (Sentinel-2)")
m.addLayer(mosaiced_sentinel1, vis_params_s1, "Sentinel-1 Mosaic (Radar)")
m.addLayer(mosaiced_sentinel2, vis_params_s2, "Sentinel-2 Mosaic (Optical)")
m.addLayer(modis_et, vis_params_modis_et, "MODIS Evapotranspiration")
#m.addLayer(pml_et, vis_params_pml_et, "PML Evapotranspiration")
m.addLayer(soil_moisture, vis_params_soil, "Soil Moisture (SMAP)")
m.addLayer(thermal, vis_params_thermal, "Thermal Infrared (Landsat)")
m.addLayer(classified, vis_params_classified, "Classified Image (K-Means)")
m.addLayer(archaeology_points, archaeology_vis, "Archaeology Points")
m.addLayer(training_points,sample_vis, "sample Points")

# Display the map
m


In [ ]:
print(training_points.size().getInfo())

In [ ]:
import csv
import numpy as np
with open("/content/Archaeological_Site_Features.csv") as infile:
    reader = csv.reader(infile, delimiter=",")
    next(reader, None)
    data3 =np.array(list(reader))

In [ ]:
#dtype1 = np.dtype([('B11', np.float32),('B12', np.float32),('B8', np.float32),('VV', np.float32), ('Elevation', np.float32),('NDVI', np.float32), ('ET', np.float32),('SM', np.float32),('ST', np.float32),('Name', np.str_),('Latitude', np.float32),('Longitude', np.float32) ,('Class', np.int32)])
#arr2 = np.zeros((len(data3),)).astype(dtype1)

In [ ]:
dtype1 = np.dtype([('B11', np.float32),('B12', np.float32),('B8', np.float32),('VV', np.float32),('Latitude', np.float32),('Longitude', np.float32),('Name', np.str_) ])
arr2 = np.zeros((len(data3),)).astype(dtype1)

In [ ]:
arr2['B11'] = data3[:, 0]
arr2['B12'] = data3[:, 1]
arr2['B8'] = data3[:, 2]
arr2['VV'] = data3[:, 3].astype(np.float32)
arr2['Latitude'] = data3[:, 4]
arr2['Longitude'] = data3[:, 5]
arr2['Name'] = data3[:, 6]

In [ ]:
arr2['B11'] = data3[:, 0]
arr2['B12'] = data3[:, 1]
arr2['B8'] = data3[:, 2]
arr2['ET'] = data3[:, 6]
arr2['NDVI'] = data3[:, 5].astype(np.float32)
arr2['ST'] = data3[:, 8].astype(np.float32)
arr2['VV'] = data3[:, 3].astype(np.float32)
arr2['Elevation'] = data3[:, 4]
arr2['SM'] = data3[:, 7]
arr2['Name'] = data3[:, 9]
arr2['Latitude'] = data3[:, 10]
arr2['Longitude'] = data3[:, 11]
arr2['Class'] = data3[:, 12]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Input,Reshape
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
# Define custom loss function (MSE + small L1 penalty)
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true)) + 0.01 * tf.reduce_sum(tf.abs(y_pred))
def r2_score(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - (SS_res / (SS_tot + tf.keras.backend.epsilon()))
# Define feature columns (exclude Name, Latitude, Longitude)
#feature_columns = ['B11', 'B12', 'B8', 'VV', 'Elevation', 'NDVI', 'ET', 'SM', 'ST']
feature_columns = ['B11', 'B12', 'B8', 'VV']
X_structured = arr2[feature_columns]  # Extract features
# Extract latitude & longitude properly from the structured array
y = np.stack([arr2['Latitude'], arr2['Longitude']], axis=1).astype(np.float32)
X = np.stack([X_structured[col] for col in feature_columns], axis=1).astype(np.float32)
X_lstm1 = X.reshape((X.shape[0], 1, X.shape[1]))
# Normalize features using MinMaxScaler
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)  # Scale features

# Normalize target (Latitude & Longitude) separately
scaler_y = MinMaxScaler(feature_range=(-1, 1))
y_scaled = scaler_y.fit_transform(y)

# Reshape X to fit LSTM input format (samples, timesteps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 1 timestep

# Build LSTM model for predicting Latitude & Longitude
from tensorflow.keras.layers import Dropout

'''model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, len(feature_columns))),
    Dropout(0.3),  # Drops 30% of neurons randomly
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(2)  # Predicts Latitude & Longitude
])'''

input_layer = Input(shape=(1, len(feature_columns)))
x = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001)))(input_layer)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)

x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)

x = LSTM(64, kernel_regularizer=l2(0.005))(x)
x = Dropout(0.2)(x)

x = Reshape((1, 64))(x)  # Ensure 3D shape
x = LSTM(32, kernel_regularizer=l2(0.001), return_sequences=False)(x)
x = Dropout(0.2)(x)

# Dense layers
x = Dense(16, activation='relu', kernel_regularizer=l2(0.001))(x)
output_layer = Dense(2)(x)  # Predicts Latitude & Longitude
# Build model
model = Model(inputs=input_layer, outputs=output_layer)


# Compile model with MSE loss (regression task)
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.Huber(delta=1.0), metrics=['mse'])#,loss=custom_loss, metrics= ['accuracy'])#['mse'])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.Huber(delta=1.0),metrics= ['mse'])#, metrics=['r2_score'])
# Summary of the model
model.summary()




In [ ]:
# Extract numeric values from the structured array
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_lstm1, y, epochs=50, batch_size=32, validation_split=0.25, callbacks=[early_stopping])
# Predict longitude & latitude for new site features
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

# Plot accuracy using MSE (lower is better)
plt.subplot(1, 2, 2)
plt.plot(history.history['mse'], label='Training mse')
#plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_mse'], label='Validation mse')
#plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epochs')
#plt.ylabel('Mean Squared Error')
plt.ylabel('mse')
plt.title('Training vs Validation accuracy')
plt.legend()

plt.show()


In [ ]:
print(history.history.keys())


In [ ]:
import csv
import numpy as np
with open("/content/Samples_random_collection4.csv") as infile:
    reader = csv.reader(infile, delimiter=",")
    next(reader, None)
    data4 =np.array(list(reader))

In [ ]:
dtype3 = np.dtype([('B11', np.float32),('B12', np.float32),('B8', np.float32), ('ET', np.float32),('NDVI', np.float32), ('ST', np.float32),('VV', np.float32),('Elevation', np.float32),('SM', np.float32) ])
arr4 = np.zeros((len(data4),)).astype(dtype3)

In [ ]:
arr4['B11'] = data4[:, 0]
arr4['B12'] = data4[:, 1]
arr4['B8'] = data4[:, 2]
arr4['ET'] = data4[:, 3]
arr4['NDVI'] = data4[:, 4].astype(np.float32)
arr4['ST'] = data4[:, 5].astype(np.float32)
arr4['VV'] = data4[:, 6].astype(np.float32)
arr4['Elevation'] = data4[:, 7]
arr4['SM'] = data4[:, 8]

In [ ]:
import pandas as pd
import numpy as np

# Define feature columns
feature_columns = ['B11', 'B12', 'B8', 'VV', 'Elevation', 'NDVI', 'ET', 'SM', 'ST']

# Prepare input data
new_data_for_scaling = np.stack([arr4[col] for col in feature_columns], axis=1).astype(np.float32)

# Normalize the new data
#new_scaled = scaler.transform(new_data_for_scaling)

# Reshape for LSTM input format
#new_lstm_ready = new_scaled.reshape((new_scaled.shape[0], 1, new_scaled.shape[1]))
new_lstm_ready = new_data_for_scaling.reshape((new_data_for_scaling.shape[0], 1, new_data_for_scaling.shape[1]))
# Make predictions (now expecting longitude & latitude)
predicted_coords_scaled = model.predict(new_lstm_ready)
predicted_coords = scaler_y.inverse_transform(predicted_coords_scaled)
# Convert predictions to a DataFrame
results_df = pd.DataFrame(new_data_for_scaling, columns=feature_columns)  # Original input features

# Add predicted longitude & latitude columns
results_df['Predicted_Longitude'] = predicted_coords[:, 1]  # First column = longitude
results_df['Predicted_Latitude'] = predicted_coords[:, 0]  # Second column = latitude

# Save results to a CSV file
results_df.to_csv('predicted_coordinates.csv', index=False)

print("Predicted latitude and longitude saved to predicted_coordinates.csv")


In [ ]:
import numpy as np

# True vs predicted coordinates
true_lat = arr2["Latitude"]
true_lon = arr2["Longitude"]
pred_lat = predicted_coords[:, 0]  # Latitude
pred_lon = predicted_coords[:, 1]  # Longitude
print('true latitude',true_lat)
print('true longitude',true_lon)
print('predi latitude',pred_lat)
print('predited longitude',pred_lon)
# Calculate absolute errors for each point
lat_errors = np.abs(true_lat - pred_lat)
lon_errors = np.abs(true_lon - pred_lon)

# Compute mean errors
mean_lat_error = np.mean(lat_errors)
mean_lon_error = np.mean(lon_errors)

print("Individual Latitude Errors:", lat_errors)
print("Mean Latitude Error:", mean_lat_error)
print("Individual Longitude Errors:", lon_errors)
print("Mean Longitude Error:", mean_lon_error)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot true locations
plt.scatter(arr2["Longitude"], arr2["Latitude"], color='green', label='True Locations', marker='o')

# Plot predicted locations
plt.scatter(predicted_coords[:, 1], predicted_coords[:, 0], color='red', label='Predicted Locations', marker='x')

# Connect true and predicted points to show displacement
#for i in range(len(true_coords)):
#    plt.plot([true_coords[i, 0], pred_coords[i, 0]], [true_coords[i, 1], pred_coords[i, 1]], color="gray", linestyle="dashed", alpha=0.5)

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("True vs. Predicted Locations of Archaeological Sites")
plt.legend()
plt.grid()
plt.show()



In [ ]:
dtype2 = np.dtype([('B11', np.float32),('B12', np.float32),('B8', np.float32), ('ET', np.float32),('NDVI', np.float32), ('ST', np.float32),('VV', np.float32),('Elevation', np.float32),('SM', np.float32),('Longitude', np.float32),('Latitude', np.float32) ])
arr3 = np.zeros((len(data4),)).astype(dtype2)

In [ ]:
arr3['B11'] = data4[:, 0]
arr3['B12'] = data4[:, 1]
arr3['B8'] = data4[:, 2]
arr3['ET'] = data4[:, 3]
arr3['NDVI'] = data4[:, 4].astype(np.float32)
arr3['ST'] = data4[:, 5].astype(np.float32)
arr3['VV'] = data4[:, 6].astype(np.float32)
arr3['Elevation'] = data4[:, 7]
arr3['SM'] = data4[:, 8]
arr3['Longitude'] = data4[:, 9]
arr3['Latitude'] = data4[:, 10]

In [ ]:
# prediction module
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
feature_columns = ['B11', 'B12', 'B8','VV', 'Elevation','NDVI','ET', 'SM', 'ST']
# Prepare input data
new_data_for_scaling = np.stack([arr3[col] for col in feature_columns], axis=1).astype(np.float32)

# Normalize the new data
new_scaled = scaler.transform(new_data_for_scaling)

# Reshape for LSTM input format
new_lstm_ready = new_scaled.reshape((new_scaled.shape[0], 1, new_scaled.shape[1]))

# Make predictions
predicted_classes = model.predict(new_lstm_ready)

# Convert predictions to a DataFrame
results_df = pd.DataFrame(new_data_for_scaling, columns=feature_columns)  # Original input features
results_df['Predicted_Class'] = predicted_classes.flatten()  # Add predictions
results_df['Longitude'] = arr3['Longitude']
results_df['Latitude'] = arr3['Latitude']
# Save results to a CSV file
results_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")

In [ ]:
# This is based on the class
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
# Define feature columns (exclude Name, Latitude, Longitude)
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true)) + 0.01 * tf.reduce_sum(tf.abs(y_pred))
feature_columns = ['B11', 'B12', 'B8','VV', 'Elevation','NDVI','ET', 'SM', 'ST']
X_structured = arr2[feature_columns]  # Extract features
y = arr2[['Class']]  # Binary target (1 for archaeology, 0 for non-archaeology)
X = np.stack([X_structured[col] for col in feature_columns], axis=1).astype(np.float32)
# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape to fit LSTM input format: (samples, timesteps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 1 timestep

# Build LSTM model for binary classification
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, len(feature_columns))),
    LSTM(32),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output: Binary classification (archaeological site or not)
])

model.compile(optimizer=Adam(learning_rate=0.0001),loss=custom_loss,metrics=['mse'])# loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
y = np.stack([arr2['Class']], axis=1).astype(np.float32)
model.fit(X_lstm, y, epochs=30, batch_size=32, validation_split=0.2)